<a href="https://colab.research.google.com/github/dantae74/machine-learning/blob/main/hans-on-ml/rnn-01-univariate-time-series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

모두를 위한 머신러닝에서 가져왔습니다.
# RNN-lot


In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from io import BytesIO
import requests

In [14]:
def generate_time_series(batch_size, n_steps):
    freq1, freq2, offset1, offset2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0,1,n_steps)
    series = 0.5 * np.sin((time-offset1) * (freq1*10+10)) # 사인곡선 1
    series += 0.2 * np.sin((time-offset2) * (freq2*20+20)) # + 사인곡선 2
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5) # + 잡음
    return series[..., np.newaxis].astype(np.float32)

In [17]:
n_step = 50
series = generate_time_series(10000, n_step+1)
x_train, y_train = series[:7000, :n_step], series[:7000, -1]
x_valid, y_valid = series[7000:9000, :n_step], series[7000:9000, -1]
x_test, y_test = series[9000:, :n_step], series[9000:, -1]

In [16]:
series.shape

(10000, 51, 1)